In [77]:
import pandas as pd
import numpy as np
import string
import scipy
import sklearn
import spacy
import nltk
import re
import os

In [78]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util

In [79]:
df = pd.read_csv(r"C:\Users\RedHat\Downloads\Tiiqu\Sample_Data_with_Keywords.csv")

In [80]:
#df

In [81]:
folder_path = r'C:\Users\RedHat\Downloads\Tiiqu\Subtopics'
subtopic_df = read_csv_files_in_folder(folder_path)

In [82]:
#subtopic_df

In [83]:
model = SentenceTransformer("BAAI/bge-small-en-v1.5")

In [84]:
df['Keywords_Emb'] = df['Keywords'].apply(lambda x: model.encode(x))

In [85]:
subtopic_df['Kywds_Emb'] = subtopic_df['Keywords'].apply(lambda x: model.encode(x))

In [86]:
def find_most_relevant_qna(subtopic_index, subtopic_df, qna_df):
    
    # Extract the vector embedding of the given subtopic index
    subtopic_index_embedding = subtopic_df.loc[subtopic_index, 'Kywds_Emb']
    
    subtopic = subtopic_df.loc[subtopic_index, 'Terminology']
    
    # Get vector embeddings for QnA pairs in the QnA DataFrame
    qna_embeddings = qna_df['Keywords_Emb'].tolist()
    
    # Compute cosine similarity between the query subtopic and all QnA pairs
    similarities = cosine_similarity([subtopic_index_embedding], qna_embeddings)
    
    # Get the indices of top 5 most similar QnA pairs
    top_indices = similarities.argsort()[0][::-1][:5]
    
    # Extract top similar QnA pairs along with their similarity scores
    top_similar_qna = qna_df.iloc[top_indices][['Question', 'Answer']]
    top_similar_qna['Relevancy Score'] = similarities[0][top_indices]

    print(subtopic)
    
    return top_similar_qna

In [87]:
subtopic_index = 200
top_similar_qna = find_most_relevant_qna(subtopic_index, subtopic_df, df)
top_similar_qna

Traveler Information Systems


,Question,Answer,Relevancy Score
2807,How can efficient data analytics help in manag...,Efficient data analytics can help in the real-...,0.887280
1771,How can a detailed digital map and real-time t...,A detailed digital map and real-time traffic a...,0.865381
2955,What is the purpose of the Real-Time Traffic P...,The purpose is to collect and analyse traffic ...,0.841836
3700,What are some applications of big data analyti...,Big data analytics can be used for traffic rou...,0.839652
2724,What is the purpose of the dynamic intelligent...,The purpose of the proposed system is to accep...,0.834814
